In [1]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, const
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive, Jigsaw
from sudoku_variants.func import rules as R


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive(), Jigsaw()]

In [2]:
import timeit


def profile(number: int):
    def inner(func):
        def wrapper(*args, **kwargs):
            def run():
                func(*args, **kwargs)

            time = timeit.timeit("run()", globals=locals(), number=number)
            print(f"Time: {time:.4f}s, {number}x | {time/number:.4f}s/time")

        return wrapper

    return inner

### AI

In [3]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(const.NUM_COL)] for _ in range(const.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = R.with_standard_rules(list(subset))

        # board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
        # sudoku = Sudoku(board, rules)
        # R.populate_initial_data(rules)
        # SudokuAI.solve(sudoku)
        # R.extract_data_from_board(rules, sudoku.board)

        run(rules)


Time: 0.0283s, 10x | 0.0028s/time
Knight
Time: 1.9804s, 10x | 0.1980s/time
King
Time: 0.0778s, 10x | 0.0078s/time
Consecutive
Time: 0.0334s, 10x | 0.0033s/time
Jigsaw
Time: 0.0285s, 10x | 0.0028s/time
Knight, King
Time: 2.2329s, 10x | 0.2233s/time
Knight, Consecutive
Time: 5.3126s, 10x | 0.5313s/time
Knight, Jigsaw
Time: 2.0501s, 10x | 0.2050s/time
King, Consecutive
Time: 0.0993s, 10x | 0.0099s/time
King, Jigsaw
Time: 0.1899s, 10x | 0.0190s/time
Consecutive, Jigsaw
Time: 0.0345s, 10x | 0.0034s/time
Knight, King, Consecutive
Time: 1.8513s, 10x | 0.1851s/time
Knight, King, Jigsaw
Time: 3.2142s, 10x | 0.3214s/time
Knight, Consecutive, Jigsaw
Time: 3.3450s, 10x | 0.3345s/time
King, Consecutive, Jigsaw
Time: 0.0481s, 10x | 0.0048s/time
Knight, King, Consecutive, Jigsaw
Time: 1.7992s, 10x | 0.1799s/time


In [4]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


rules = [Orthogonal(), SubBoard()]
for to_erase in [30, 40, 50, 60]:
    print(to_erase)
    run(rules, to_erase)

30
Time: 0.1472s, 10x | 0.0147s/time
40
Time: 0.3808s, 10x | 0.0381s/time
50
Time: 4.3327s, 10x | 0.4333s/time
60
Time: 167.8731s, 10x | 16.7873s/time


In [5]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()], max_erased=40)
    if sudoku is not None:
        sudoku.show()


run()

Rules: Orthogonal, SubBoard, Knight, King, Consecutive
+-------+-------+-------+
| * 5 * | * * 7 | 1 * * |
| 3 1 * | 8 * 4 | 6 2 * |
| * * 2 | * 1 9 | 8 * * |
+-------+-------+-------+
| 4 * * | 7 * 2 | 3 1 * |
| * 3 1 | 4 * 5 | * * * |
| 2 * * | * * 1 | 4 * * |
+-------+-------+-------+
| 5 4 8 | * 7 6 | * 3 * |
| 1 9 3 | * 4 * | * 7 6 |
| 6 * * | 1 * 3 | * 4 * |
+-------+-------+-------+
Time: 0.0622s, 1x | 0.0622s/time
